VAE FOR SONG RECOMMENDATION

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics.pairwise import cosine_similarity


data_file = "/content/drive/MyDrive/CS573_Final_Project-main/datasets/user_month_datasets/user1_1month_listening_history.csv"
data = pd.read_csv(data_file)

# Select numerical columns for VAE
numerical_columns = [
    "duration (ms)", "danceability", "energy", "loudness",
    "speechiness", "acousticness", "instrumentalness", "liveness",
    "valence", "tempo", "spec_rate"
]
features = data[numerical_columns].values

# Scaling the features
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

X_train, X_temp = train_test_split(scaled_features, test_size=0.3, random_state=42)
X_val, X_test = train_test_split(X_temp, test_size=0.5, random_state=42)


class SongDataset(Dataset):
    def __init__(self, data):
        self.data = torch.tensor(data, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

train_dataset = SongDataset(X_train)
val_dataset = SongDataset(X_val)
test_dataset = SongDataset(X_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)



In [6]:
# VAE Model
class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(VAE, self).__init__()
        # Encoder
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2_mean = nn.Linear(hidden_dim, latent_dim)
        self.fc2_logvar = nn.Linear(hidden_dim, latent_dim)
        # Decoder
        self.fc3 = nn.Linear(latent_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, input_dim)

    def encode(self, x):
        h = torch.relu(self.fc1(x))
        mean = self.fc2_mean(h)
        logvar = self.fc2_logvar(h)
        return mean, logvar

    def reparameterize(self, mean, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mean + eps * std

    def decode(self, z):
        h = torch.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h))

    def forward(self, x):
        mean, logvar = self.encode(x)
        z = self.reparameterize(mean, logvar)
        reconstruction = self.decode(z)
        return reconstruction, mean, logvar

# Loss function and optimizer (KL DIVERGENCE)
def vae_loss(reconstructed, original, mean, logvar):
    recon_loss = nn.MSELoss()(reconstructed, original)
    kl_div = -0.5 * torch.sum(1 + logvar - mean.pow(2) - logvar.exp()) / original.size(0)
    return recon_loss + kl_div

In [7]:
input_dim = len(numerical_columns)
hidden_dim = 64
latent_dim = 16
learning_rate = 1e-3
num_epochs = 50

model = VAE(input_dim, hidden_dim, latent_dim)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        reconstructed, mean, logvar = model(batch)
        loss = vae_loss(reconstructed, batch, mean, logvar)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")

#  Generating recommendations
def recommend_similar_songs(song_features, top_k=10):
    model.eval()
    with torch.no_grad():
        # Encode input song features into latent space
        song_tensor = torch.tensor(song_features, dtype=torch.float32)
        mean, logvar = model.encode(song_tensor)
        latent_space = model.reparameterize(mean, logvar).numpy()

        # Compute similarity in latent space
        all_latents = []
        song_indices = []  # Keep track of original indices for recommendations
        for idx, data_batch in enumerate(train_loader):
            mean, logvar = model.encode(data_batch)
            z = model.reparameterize(mean, logvar)
            all_latents.append(z.numpy())
            song_indices.extend(range(idx * train_loader.batch_size,
                                      idx * train_loader.batch_size + len(data_batch)))

        all_latents = np.concatenate(all_latents, axis=0)

        # Compute similarities
        similarities = cosine_similarity(latent_space, all_latents)
        top_indices = np.argsort(-similarities, axis=1)[:, :top_k].flatten()

        # Map back to original song indices
        original_indices = [song_indices[i] for i in top_indices]
        return data.iloc[original_indices]


song_index = 0  # Index of the song to recommend based on
input_song_features = scaled_features[song_index].reshape(1, -1)
recommended_songs = recommend_similar_songs(input_song_features)
print(recommended_songs['uri'])


Epoch 1, Loss: 0.2022225129253724
Epoch 2, Loss: 0.08875186434563469
Epoch 3, Loss: 0.07414564577972188
Epoch 4, Loss: 0.06909756976015427
Epoch 5, Loss: 0.06615586206316948
Epoch 6, Loss: 0.06454910798107877
Epoch 7, Loss: 0.06347703626927208
Epoch 8, Loss: 0.06281004079124507
Epoch 9, Loss: 0.062298126299591625
Epoch 10, Loss: 0.061734111870036405
Epoch 11, Loss: 0.06147115589941249
Epoch 12, Loss: 0.061315379160292005
Epoch 13, Loss: 0.06102457094718428
Epoch 14, Loss: 0.06086958638008903
Epoch 15, Loss: 0.06059109726372887
Epoch 16, Loss: 0.060362689635332895
Epoch 17, Loss: 0.060691792956169915
Epoch 18, Loss: 0.06039837879293105
Epoch 19, Loss: 0.060134528533500785
Epoch 20, Loss: 0.06010258964755956
Epoch 21, Loss: 0.06011613305000698
Epoch 22, Loss: 0.06037151550545412
Epoch 23, Loss: 0.06010124503689654
Epoch 24, Loss: 0.059874408385332895
Epoch 25, Loss: 0.05998741506653674
Epoch 26, Loss: 0.060032986761892546
Epoch 27, Loss: 0.06000216217602
Epoch 28, Loss: 0.059763136593734

Extraction of Song Names using Spotify API

In [8]:
!pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Authentication with Spotify API
client_id = "75d0ab19dcdc4db7821a27bf07df72a0"
client_secret = "f64897e446834d7cb83b1c90916242df"

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

# Function to get song names from URIs
def get_song_names(uris):
    song_names = []
    for uri in uris:
        try:
            track = sp.track(uri)
            song_names.append({
                "name": track['name'],
                "artist": track['artists'][0]['name'],
                "album": track['album']['name'],
                "uri": uri
            })
        except Exception as e:
            print(f"Error fetching details for URI {uri}: {e}")
            song_names.append({"name": "Unknown", "artist": "Unknown", "album": "Unknown", "uri": uri})
    return song_names


recommended_uris = recommended_songs['uri']
print(recommended_songs)
song_metadata = get_song_names(recommended_uris)
for song in song_metadata:
    print(f"Name: {song['name']}, Artist: {song['artist']}, Album: {song['album']}, URI: {song['uri']}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 6.7 MB/s eta 0:00:00
      duration (ms)  danceability  energy  loudness  speechiness  \
12         402587.0      0.559816   0.270  0.297548     0.313907   
761        214379.0      0.649780   0.529  0.601850     0.582207   
91         229680.0      0.356324   0.414  0.427953     0.184768   
963        248750.0      0.631573   0.200  0.183293     0.187417   
332        222179.0      0.801864   0.158  0.251817     0.515894   
571        217088.0      0.647638   0.787  0.679439     0.211921   
964         85848.0      0.574810   0.318  0.289951     0.205298   
396        310560.0      0.595159   0.762  0.599684     0.174172   
398        240053.0      0.846846   0.293  0.555164     0.462914   
1007       172853.0      0.617650   0.706  0.647324     0.218543   

      acousticness  instrumentalness  liveness  valence     tempo  \
12        0.864458          0.872000  0.248227    0.451  0.545177   
761       0.494980          0.000

KeyboardInterrupt: 

#In this section we do:
*VAE Training:* Train 10 separate VAEs, one for each user. <br>
*Generation of Recommendations:* Generate song recommendations for each VAE (each user). <br>
*Saving Outputs:* Save models and recommendations for evaluation. <br>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [9]:
import os
import json
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

In [16]:


USER_DATASETS_FOLDER = "/content/drive/MyDrive/CS573_Final_Project-main/datasets/user_month_datasets/"
MODELS_FOLDER = "/content/drive/MyDrive/VAE/vae_models/"
OUTPUT_FILE = "/content/drive/MyDrive/VAE/vae_recommendations/vae_recommendations.json"
#os.makedirs(MODELS_FOLDER, exist_ok=True)


In [17]:
# VAE model definition
class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2_mean = nn.Linear(hidden_dim, latent_dim)
        self.fc2_logvar = nn.Linear(hidden_dim, latent_dim)
        self.fc3 = nn.Linear(latent_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, input_dim)

    def encode(self, x):
        h = torch.relu(self.fc1(x))
        mean = self.fc2_mean(h)
        logvar = self.fc2_logvar(h)
        return mean, logvar

    def reparameterize(self, mean, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mean + eps * std

    def decode(self, z):
        h = torch.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h))

    def forward(self, x):
        mean, logvar = self.encode(x)
        z = self.reparameterize(mean, logvar)
        reconstruction = self.decode(z)
        return reconstruction, mean, logvar


class SongDataset(Dataset):
    def __init__(self, data):
        self.data = torch.tensor(data, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Loss function with kl divergence
def vae_loss(reconstructed, original, mean, logvar):
    recon_loss = nn.MSELoss()(reconstructed, original)
    kl_div = -0.5 * torch.sum(1 + logvar - mean.pow(2) - mean.pow(2) - logvar.exp()) / original.size(0)
    return recon_loss + kl_div


In [18]:

# Training function for each VAE
def train_vae(features, input_dim, hidden_dim, latent_dim, num_epochs, learning_rate):
    dataset = SongDataset(features)
    dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
    model = VAE(input_dim, hidden_dim, latent_dim)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in dataloader:
            optimizer.zero_grad()
            reconstructed, mean, logvar = model(batch)
            loss = vae_loss(reconstructed, batch, mean, logvar)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
    return model

# Recommendation function
def recommend_similar_songs(model, features, data, top_k=10):
    model.eval()
    with torch.no_grad():
        song_tensor = torch.tensor(features, dtype=torch.float32)
        mean, logvar = model.encode(song_tensor)
        latent_space = model.reparameterize(mean, logvar).numpy()
        similarities = cosine_similarity(latent_space, latent_space)
        recommendations = np.argsort(-similarities, axis=1)[:, :top_k]
        return [
            data['uri'].iloc[idx] for idx in recommendations[0]
        ]

# Main function to process users
def process_users(hidden_dim=64, latent_dim=16, num_epochs=50, learning_rate=1e-3, max_users=10, top_k=10):
    playlists = {}
    user_count = 0
    for file in os.listdir(USER_DATASETS_FOLDER):
        if file.endswith(".csv") and user_count < max_users:
            user_id = file.split("_")[0].replace("user", "")
            print(f"Processing User {user_id}...")

            # Loading and preprocessing user data
            user_data = pd.read_csv(os.path.join(USER_DATASETS_FOLDER, file))
            numerical_columns = [
                "duration (ms)", "danceability", "energy", "loudness",
                "speechiness", "acousticness", "instrumentalness", "liveness",
                "valence", "tempo", "spec_rate"
            ]
            features = user_data[numerical_columns].values
            scaler = MinMaxScaler()
            scaled_features = scaler.fit_transform(features)

            # Train a single VAE for each user
            input_dim = len(numerical_columns)
            print(f"Training VAE for User {user_id}...")
            model = train_vae(scaled_features, input_dim, hidden_dim, latent_dim, num_epochs, learning_rate)

            # Saving of the model
            model_path = os.path.join(MODELS_FOLDER, f"user_{user_id}_vae.pth")
            torch.save(model.state_dict(), model_path)

            # Generating recommendations for each user
            recommended_uris = recommend_similar_songs(model, scaled_features, user_data, top_k=top_k)
            playlists[user_id] = recommended_uris

            user_count += 1

    # Save all recommendations to a single JSON file
    with open(OUTPUT_FILE, "w") as f:
        json.dump(playlists, f)

    print(f"All recommendations saved to {OUTPUT_FILE}")

process_users(top_k=10)  # top_k can be modified to change the number of recommendations per user


Processing User 7...
Training VAE for User 7...
Processing User 8...
Training VAE for User 8...
Processing User 6...
Training VAE for User 6...
Processing User 9...
Training VAE for User 9...
Processing User 5...
Training VAE for User 5...
Processing User 2...
Training VAE for User 2...
Processing User 10...
Training VAE for User 10...
Processing User 3...
Training VAE for User 3...
Processing User 1...
Training VAE for User 1...
Processing User 4...
Training VAE for User 4...
All recommendations saved to /content/drive/MyDrive/VAE/vae_recommendations/vae_recommendations.json
